# Student Performance

## Данные
Рассмотрим набор данных [Student Performance Data Set](https://archive.ics.uci.edu/ml/datasets/student+performance): данные об успеваемости учащихся.

Имеется 2 набора данных: математика (395 студентов) и португальский язык (649 студентов), но первый почти полностью лежит во втором, поэтому будем рассматривать только данные об изучающих португальский язык.

Для каждого студента имеется 30 признаков + 3 целевых признака (G1, G2, G3) - оценка по курсу (за первую часть курса, за вторую часть курса и итоговая).

## Задание
Нужно решить задачу предсказания целевого признака G3 итоговой оценки студента по курсу. G3 сильно коррелирует с G1 и G2, поэтому пользоваться этими признаками нельзя. Для оценки качества моделей использовать кросс-валидацию с 10 фолдами, функция потерь - Mean Absolute Error $L(y_{true}, y_{pred}) = \frac{1}{l}\sum_{i=1}^l |y_{true, i} - y_{pred, i}|$. Ваша задача - получить наименьшую ошибку на кросс-валидации, для этого поэкспериментируйте с разными моделями машинного обучения и разными признаковыми описаними объектов. Результаты оформите в виде Jupyter Notebook. Весь код должен без проблем запускаться и отрабатывать. Также оформите некоторые выводы (какие методы Вы использовали, что получилось, какой метод лучший).

Замечения:
1. Для загрузки данных можно использовать pandas, для машинного обучения - sklearn.
2. Для кросс-валидации можно использовать модуль sklearn.model_selection. Для воспроизводимости результатов явно инициализируйте генератор случайных чисел (разберитесь, как).
3. Обратите внимание, что многие признаки в наборе данных - не вещественные, а категориальные (categorical) или порядковые (ordinal). О том, как работать с такими признаками, можно начать читать [здесь](https://dyakonov.org/2016/08/03/python-%d0%ba%d0%b0%d1%82%d0%b5%d0%b3%d0%be%d1%80%d0%b8%d0%b0%d0%bb%d1%8c%d0%bd%d1%8b%d0%b5-%d0%bf%d1%80%d0%b8%d0%b7%d0%bd%d0%b0%d0%ba%d0%b8/).
4. Обратите внимание, что целевой признак - целые числа от 0 до 20.
5. Обратите внимание, что не все алгоритмы обучения способны работать с L1-loss. Но не факт, что их нужно обучать с L1-loss.

In [68]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder

In [66]:
df = pd.read_csv('student-por.csv', sep=';')

In [9]:
df.head(10).T

,0,1,2,3,4,5,6,7,8,9
school,GP,GP,GP,GP,GP,GP,GP,GP,GP,GP
sex,F,F,F,F,F,M,M,F,M,M
age,18,17,15,15,16,16,16,17,15,15
address,U,U,U,U,U,U,U,U,U,U
famsize,GT3,GT3,LE3,GT3,GT3,LE3,LE3,GT3,LE3,GT3
Pstatus,A,T,T,T,T,T,T,A,A,T
Medu,4,1,1,4,3,4,2,4,3,3
Fedu,4,1,1,2,3,3,2,4,2,4
Mjob,at_home,at_home,at_home,health,other,services,other,other,services,other
Fjob,teacher,other,other,services,other,other,other,teacher,other,other


In [20]:
df2 = pd.get_dummies(df, columns=['Medu', 'Fedu', 'traveltime', 'studytime', 'school', 'sex',\
                            'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', \
                            'guardian', 'schoolsup', 'famsup', 'paid', 'activities', \
                            'nursery', 'higher', 'internet', 'romantic'])

In [78]:
category = ['Medu', 'Fedu', 'traveltime', 'studytime', 'school', 'sex',\
                            'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', \
                            'guardian', 'schoolsup', 'famsup', 'paid', 'activities', \
                            'nursery', 'higher', 'internet', 'romantic']

In [74]:
X = df.drop(['G1', 'G2', 'G3'], axis=1)
X_L = X.copy()

In [76]:
Y = df['G3']

In [79]:
for c in category:
    le = LabelEncoder()
    le.fit(X_L[c])
    X_L = le.transform(X_L[c])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [21]:
df2.head().T

,0,1,2,3,4
age,18,17,15,15,16
failures,0,0,0,0,0
famrel,4,5,4,3,4
freetime,3,3,3,2,3
goout,4,3,2,2,2
Dalc,1,1,2,1,1
Walc,1,1,3,1,2
health,3,3,3,5,5
absences,4,2,6,0,0
G1,0,9,12,14,11


In [42]:
lr = Ridge(random_state=123890)

In [53]:
gr = GridSearchCV(estimator=lr, param_grid={'alpha': np.arange(0.001, 1, 0.05)},\
                  n_jobs=-1, verbose=1, cv=10, scoring='neg_mean_absolute_error',\
                 return_train_score=True)

In [54]:
gr.fit(X, Y)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    0.9s finished
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=123890, solver='auto', tol=0.001),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': array([0.001, 0.051, 0.101, 0.151, 0.201, 0.251, 0.301, 0.351, 0.401,
       0.451, 0.501, 0.551, 0.601, 0.651, 0.701, 0.751, 0.801, 0.851,
       0.901, 0.951])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_absolute_error', verbose=1)

In [55]:
gr.best_score_

-2.0642985927560247

In [58]:
df3 = pd.DataFrame(gr.cv_results_)

In [64]:
df3['mean_test_score'].T

0    -2.067863
1    -2.067664
2    -2.067465
3    -2.067268
4    -2.067073
5    -2.066879
6    -2.066687
7    -2.066496
8    -2.066307
9    -2.066119
10   -2.065931
11   -2.065746
12   -2.065561
13   -2.065377
14   -2.065195
15   -2.065014
16   -2.064833
17   -2.064654
18   -2.064476
19   -2.064299
Name: mean_test_score, dtype: float64

In [ ]:
lass
elasticnet
boosting
randomforest